In [2]:
# 导入系统库
from functools import partial
import os
import time
# 导入Paddle库
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle import inference

# 导入PaddleNLP相关的库
import paddlenlp as ppnlp
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.datasets import load_dataset, MapDataset
from paddlenlp.transformers import LinearDecayWithWarmup

# 忽略所有警告
import warnings
warnings.filterwarnings("ignore")

In [4]:
# 数据读取逻辑
def read_simcse_text(data_path):
    """Reads data."""
    with open(data_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i == 0:
                continue
            data = line.rstrip()
            # 这里的text_a和text_b是一样的
            yield {'text_a': data, 'text_b': data}


# 数据集路径
train_set_file = './data/train_demo.csv'
train_ds = load_dataset(read_simcse_text, data_path=train_set_file, lazy=False)

# 在训练神经网络之前，我们需要构建小批量的数据，所以需要借助Dataloader

In [5]:
# 明文数据 -> ID 序列训练数据
def create_dataloader(dataset,
                      mode='train',
                      batch_size=1,
                      batchify_fn=None,
                      trans_fn=None):
    if trans_fn:
        dataset = dataset.map(trans_fn)

    shuffle = True if mode == 'train' else False
    if mode == 'train':
        # 分布式批采样器加载数据的一个子集。
        # 每个进程可以传递给DataLoader一个DistributedBatchSampler的实例，每个进程加载原始数据的一个子集。
        batch_sampler = paddle.io.DistributedBatchSampler(
            dataset, batch_size=batch_size, shuffle=shuffle)
    else:
        # 批采样器的基础实现，
        # 用于 paddle.io.DataLoader 中迭代式获取mini-batch的样本下标数组，数组长度与 batch_size 一致。
        batch_sampler = paddle.io.BatchSampler(
            dataset, batch_size=batch_size, shuffle=shuffle)
    # 组装mini-batch
    return paddle.io.DataLoader(
        dataset=dataset,
        batch_sampler=batch_sampler,
        collate_fn=batchify_fn,
        return_list=True)


def convert_example(example, tokenizer, max_seq_length=512, do_evalute=False):
    result = []

    for key, text in example.items():
        if 'label' in key:
            # do_evaluate
            result += [example['label']]
        else:
            # do_train
            encoded_inputs = tokenizer(text=text, max_seq_len=max_seq_length)
            input_ids = encoded_inputs["input_ids"]
            token_type_ids = encoded_inputs["token_type_ids"]
            result += [input_ids, token_type_ids]

    return result


In [6]:
# 语义索引的维度最大为64，可以根据自己的情况调节长度
max_seq_length = 64
# 根据经验 batch_size越大效果越好
batch_size = 32
tokenizer = ppnlp.transformers.ErnieTokenizer.from_pretrained('ernie-3.0-medium-zh')
# 给convert_example赋予默认的值，如tokenizer，max_seq_length
trans_func = partial(
    convert_example,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length)
# [pad]对齐的函数
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype='int64'),  # query_input
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype='int64'),  # query_segment
    Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype='int64'),  # title_input
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype='int64'),  # tilte_segment
): [data for data in fn(samples)]

# 构建训练的Dataloader
train_data_loader = create_dataloader(
    train_ds,
    mode='train',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func)

[2024-06-02 11:20:43,923] [    INFO] - Already cached C:\Users\kitx86\.paddlenlp\models\ernie-3.0-medium-zh\ernie_3.0_medium_zh_vocab.txt
[2024-06-02 11:20:43,948] [    INFO] - tokenizer config file saved in C:\Users\kitx86\.paddlenlp\models\ernie-3.0-medium-zh\tokenizer_config.json
[2024-06-02 11:20:43,948] [    INFO] - Special tokens file saved in C:\Users\kitx86\.paddlenlp\models\ernie-3.0-medium-zh\special_tokens_map.json


#### 模型构建

接下来搭建SimCSE模型，主要部分是用query和title分别得到embedding向量，然后计算余弦相似度。

![](https://ai-studio-static-online.cdn.bcebos.com/fa84f7db963c4efd82b971f3ef477c070e5300acee3e4e788b4a6c56dd31c003)


上图是SimCSE的原理图，SimCSE主要是通过dropout来把同一个句子变成正样本（做两次前向，但是dropout有随机因素，所以产生的向量不一样，但是本质上还是表示的是同一句话），把一个batch里面其他的句子变成负样本的。

In [7]:
class SimCSE(nn.Layer):
    def __init__(self,
                 pretrained_model,
                 dropout=None,
                 margin=0.0,
                 scale=20,
                 output_emb_size=None):

        super().__init__()

        self.ptm = pretrained_model
        # 显式的加一个dropout来控制
        self.dropout = nn.Dropout(dropout if dropout is not None else 0.1)

        # if output_emb_size is greater than 0, then add Linear layer to reduce embedding_size,
        # 考虑到性能和效率，我们推荐把output_emb_size设置成256
        # 向量越大，语义信息越丰富，但消耗资源越多
        self.output_emb_size = output_emb_size
        if output_emb_size > 0:
            weight_attr = paddle.ParamAttr(
                initializer=nn.initializer.TruncatedNormal(std=0.02))
            self.emb_reduce_linear = paddle.nn.Linear(
                768, output_emb_size, weight_attr=weight_attr)

        self.margin = margin
        # 为了使余弦相似度更容易收敛，我们选择把计算出来的余弦相似度扩大scale倍，一般设置成20左右
        self.sacle = scale

    # 加入jit注释能够把该提取向量的函数导出成静态图
    # 对应input_id,token_type_id两个
    @paddle.jit.to_static(input_spec=[paddle.static.InputSpec(shape=[None, None], dtype='int64'),
                                      paddle.static.InputSpec(shape=[None, None], dtype='int64')])
    def get_pooled_embedding(self,
                             input_ids,
                             token_type_ids=None,
                             position_ids=None,
                             attention_mask=None,
                             with_pooler=True):

        # Note: cls_embedding is poolerd embedding with act tanh
        sequence_output, cls_embedding = self.ptm(input_ids, token_type_ids,
                                                  position_ids, attention_mask)

        if with_pooler == False:
            cls_embedding = sequence_output[:, 0, :]

        if self.output_emb_size > 0:
            cls_embedding = self.emb_reduce_linear(cls_embedding)
        cls_embedding = self.dropout(cls_embedding)
        # https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/functional/normalize_cn.html
        cls_embedding = F.normalize(cls_embedding, p=2, axis=-1)
        return cls_embedding

    def forward(self,
                query_input_ids,
                title_input_ids,
                query_token_type_ids=None,
                query_position_ids=None,
                query_attention_mask=None,
                title_token_type_ids=None,
                title_position_ids=None,
                title_attention_mask=None):

        # 第 1 次编码: 文本经过无监督语义索引模型编码后的语义向量
        # [N, 768]
        query_cls_embedding = self.get_pooled_embedding(
            query_input_ids, query_token_type_ids, query_position_ids,
            query_attention_mask)

        # 第 2 次编码: 文本经过无监督语义索引模型编码后的语义向量
        # [N, 768]
        title_cls_embedding = self.get_pooled_embedding(
            title_input_ids, title_token_type_ids, title_position_ids,
            title_attention_mask)

        # 相似度矩阵: [N, N]
        cosine_sim = paddle.matmul(
            query_cls_embedding, title_cls_embedding, transpose_y=True)

        # substract margin from all positive samples cosine_sim()
        # 填充self.margin值，比如margin为0.2，query_cls_embedding.shape[0]=2
        # margin_diag: [0.2,0.2]
        margin_diag = paddle.full(
            shape=[query_cls_embedding.shape[0]],
            fill_value=self.margin,
            dtype=paddle.get_default_dtype())
        # input paddle.diag(margin_diag): [[0.2,0],[0,0.2]]
        # input cosine_sim : [[1.0,0.6],[0.6,1.0]]
        # output cosine_sim: [[0.8,0.6],[0.6,0.8]]
        cosine_sim = cosine_sim - paddle.diag(margin_diag)

        # scale cosine to ease training converge
        cosine_sim *= self.sacle

        # 转化成多分类任务: 对角线元素是正例，其余元素为负例
        # labels : [0,1,2,3]
        labels = paddle.arange(0, query_cls_embedding.shape[0], dtype='int64')
        # labels : [[0],[1],[2],[3]]
        labels = paddle.reshape(labels, shape=[-1, 1])

        # 交叉熵损失函数
        loss = F.cross_entropy(input=cosine_sim, label=labels)
        return loss

In [8]:
# 训练配置：关键参数
scale = 20  # 推荐值: 10 ~ 30
margin = 0.1  # 推荐值: 0.0 ~ 0.2
# SimCSE的dropout的参数，也可以使用预训练语言模型默认的dropout参数
dropout = 0.2
# 向量映射的维度，默认的输出是768维，推荐通过线性层映射成256维
output_emb_size = 256
# 训练的epoch数目
epochs = 1
weight_decay = 0.0
# 学习率
learning_rate = 5E-5
warmup_proportion = 0.0

#### 加载预训练模型
1. 加载预训练模型 ERNIE 3.0-Medium 进行热启
2. 定义优化器 AdamOptimizer

In [9]:
# 设置 ERNIE-3.0-Medium-zh 预训练模型
model_name_or_path = 'ernie-3.0-medium-zh'
pretrained_model = ppnlp.transformers.ErnieModel.from_pretrained(
    model_name_or_path,
    hidden_dropout_prob=dropout,
    attention_probs_dropout_prob=dropout)
print("loading model from {}".format(model_name_or_path))

# 实例化SimCSE，SimCSE使用的Encoder是ERNIE-3.0-Medium-zh
model = SimCSE(
    pretrained_model,
    margin=margin,
    scale=scale,
    output_emb_size=output_emb_size)
# 训练的总步数
num_training_steps = len(train_data_loader) * epochs
# warmpup操作，学习率先上升后下降
lr_scheduler = LinearDecayWithWarmup(learning_rate, num_training_steps,
                                     warmup_proportion)

# Generate parameter names needed to perform weight decay.
# All bias and LayerNorm parameters are excluded.
decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]
# 设置优化器
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params)

[2024-06-02 11:20:55,833] [    INFO] - Already cached C:\Users\kitx86\.paddlenlp\models\ernie-3.0-medium-zh\model_state.pdparams
[2024-06-02 11:20:55,833] [    INFO] - Loading weights file model_state.pdparams from cache at C:\Users\kitx86\.paddlenlp\models\ernie-3.0-medium-zh\model_state.pdparams
[2024-06-02 11:20:56,193] [    INFO] - Loaded weights file from disk, setting weights to model.
[2024-06-02 11:21:02,933] [ WARNING] - Some weights of the model checkpoint at ernie-3.0-medium-zh were not used when initializing ErnieModel: ['ernie.encoder.layers.6.linear1.bias', 'ernie.encoder.layers.6.linear2.weight', 'ernie.encoder.layers.6.self_attn.out_proj.bias', 'ernie.encoder.layers.6.self_attn.q_proj.weight', 'ernie.encoder.layers.6.norm1.bias', 'ernie.encoder.layers.6.self_attn.v_proj.bias', 'ernie.encoder.layers.6.linear2.bias', 'ernie.encoder.layers.6.self_attn.out_proj.weight', 'ernie.encoder.layers.6.norm2.weight', 'ernie.encoder.layers.6.self_attn.k_proj.bias', 'ernie.encoder.lay

loading model from ernie-3.0-medium-zh


#### 模型训练

上面的训练配置完毕以后，下面就可以开始训练了。

In [ ]:
save_dir = 'checkpoint'
save_steps = 100
time_start = time.time()
global_step = 0
tic_train = time.time()
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        query_input_ids, query_token_type_ids, title_input_ids, title_token_type_ids = batch
        # 其中query和title为同一条数据
        loss = model(
            query_input_ids=query_input_ids,
            title_input_ids=title_input_ids,
            query_token_type_ids=query_token_type_ids,
            title_token_type_ids=title_token_type_ids)
        # 每隔10个step进行打印日志
        global_step += 1
        if global_step % 10 == 0:
            print("global step %d, epoch: %d, batch: %d, loss: %.5f, speed: %.2f step/s"
                  % (global_step, epoch, step, loss,
                     10 / (time.time() - tic_train)))
            tic_train = time.time()
        # 反向
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()
        # 每隔save_steps保存模型
        if global_step % save_steps == 0:
            save_path = os.path.join(save_dir, "model_%d" % (global_step))
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            save_param_path = os.path.join(save_path, 'model_state.pdparams')
            paddle.save(model.state_dict(), save_param_path)
            tokenizer.save_pretrained(save_path)
time_end = time.time()
print('totally cost {} seconds'.format(time_end - time_start))

#### 模型预测

由于本项目使用的demo数据，在预测部分为了保证效果，我们使用已经用全量数据训练好的模型，首先下载训练好的SimCSE模型，然后进行解压

In [10]:
from NeuralSearch.utils.data import convert_example_test

# 加载预训练好的无监督语义索引模型 SimCSE
params_path='./pretrained/model_20000/model_state.pdparams'
state_dict = paddle.load(params_path)
model.set_dict(state_dict)
# 定义两条文本数据
test_data = ['国有企业引入非国有资本对创新绩效的影响——基于制造业国有上市公司的经验证据', '语义检索相关的论文']
# 给convert_example_test赋予默认值
test_func = partial(
        convert_example_test,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length)
# pad对齐操作
test_batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype='int64'),  # text_input
        Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype='int64'),  # text_segment
    ): [data for data in fn(samples)]

# conver_example function's input must be dict
corpus_ds = MapDataset(test_data)
# 构造Dataloader
corpus_data_loader = create_dataloader(
        corpus_ds,
        mode='predict',
        batch_size=batch_size,
        batchify_fn=test_batchify_fn,
        trans_fn=test_func)

all_embeddings = []
# 切换成eval模式，固定住 dropout
model.eval()
# 预测的时候不保存梯度
with paddle.no_grad():
    for batch_data in corpus_data_loader:
        input_ids, token_type_ids = batch_data
        input_ids = paddle.to_tensor(input_ids, dtype='int64')
        token_type_ids = paddle.to_tensor(token_type_ids, dtype='int64')
        # 抽取向量
        text_embeddings = model.get_pooled_embedding(input_ids, token_type_ids)
        all_embeddings.append(text_embeddings)

text_embedding=all_embeddings[0]
print(text_embedding.shape)
print(text_embedding.numpy())

[2, 256]
[[-2.38936171e-02 -1.86179597e-02 -3.87326293e-02 -2.14510243e-02
   5.80730699e-02 -7.30442628e-02  4.47707027e-02 -4.79865447e-03
  -1.11777507e-01  3.06986496e-02 -7.58780241e-02 -1.39245540e-02
  -3.18336859e-02  7.34863579e-02 -6.20127991e-02 -7.75121376e-02
  -2.94753723e-02 -1.10602945e-01  1.43525824e-02  7.38698058e-03
  -6.20281848e-04 -1.13605276e-01  5.09278849e-03  1.68028884e-02
  -9.10069644e-02 -4.97759227e-03  9.68626812e-02 -4.01212536e-02
  -4.31677513e-02 -6.29915670e-02 -2.77599022e-02 -1.28067151e-01
   4.59447299e-04  1.12159485e-02 -2.34453306e-02 -1.29382834e-02
  -9.12685394e-02  8.15406442e-02 -7.75527675e-03  3.35520543e-02
  -7.50929788e-02 -6.48786947e-02  8.32832009e-02 -3.95188704e-02
   1.33434922e-01 -7.62892142e-02  2.10594863e-01  9.67187062e-02
  -4.11144681e-02  5.90532506e-03  2.87344866e-02 -1.25124678e-01
  -9.30405185e-02 -1.03214726e-01  1.13968330e-03 -1.43238613e-02
   9.50518623e-02 -1.65595077e-02 -4.30812314e-02  5.52676395e-02
 

### 模型部署

模型部署首先需要把模型转换成静态图模型。

In [ ]:
output_path='./output/recall'
model.eval()
# Convert to static graph with specific input description
model = paddle.jit.to_static(
        model,
        input_spec=[
            paddle.static.InputSpec(
                shape=[None, None], dtype="int64"),  # input_ids
            paddle.static.InputSpec(
                shape=[None, None], dtype="int64")  # segment_ids
        ])
# Save in static graph model.
save_path = os.path.join(output_path, "inference")
print(save_path)
paddle.jit.save(model, save_path)

In [ ]:
from NeuralSearch.utils.data import convert_example_recall_infer
from scipy.special import softmax
from scipy import spatial

class RecallPredictor(object):
    def __init__(self,
                 model_dir,
                 device="gpu",
                 max_seq_length=128,
                 batch_size=32,
                 use_tensorrt=False,
                 precision="fp32",
                 cpu_threads=10,
                 enable_mkldnn=False):
        self.max_seq_length = max_seq_length
        self.batch_size = batch_size

        model_file = model_dir + "/output/recall/inference.get_pooled_embedding.pdmodel"
        params_file = model_dir + "/output/recall/inference.get_pooled_embedding.pdiparams"
        if not os.path.exists(model_file):
            raise ValueError("not find model file path {}".format(model_file))
        if not os.path.exists(params_file):
            raise ValueError("not find params file path {}".format(params_file))
        config = paddle.inference.Config(model_file, params_file)

        if device == "gpu":
            # set GPU configs accordingly
            # such as intialize the gpu memory, enable tensorrt
            config.enable_use_gpu(100, 0)
            precision_map = {
                "fp16": inference.PrecisionType.Half,
                "fp32": inference.PrecisionType.Float32,
                "int8": inference.PrecisionType.Int8
            }
            precision_mode = precision_map[precision]

            if use_tensorrt:
                config.enable_tensorrt_engine(
                    max_batch_size=batch_size,
                    min_subgraph_size=30,
                    precision_mode=precision_mode)
        elif device == "cpu":
            # set CPU configs accordingly,
            # such as enable_mkldnn, set_cpu_math_library_num_threads
            config.disable_gpu()
            if enable_mkldnn:
                # cache 10 different shapes for mkldnn to avoid memory leak
                config.set_mkldnn_cache_capacity(10)
                config.enable_mkldnn()
            config.set_cpu_math_library_num_threads(cpu_threads)
        elif device == "xpu":
            # set XPU configs accordingly
            config.enable_xpu(100)

        config.switch_use_feed_fetch_ops(False)
        self.predictor = paddle.inference.create_predictor(config)
        self.input_handles = [
            self.predictor.get_input_handle(name)
            for name in self.predictor.get_input_names()
        ]
        self.output_handle = self.predictor.get_output_handle(
            self.predictor.get_output_names()[0])



    def extract_embedding(self, data, tokenizer):
        """
        Predicts the data labels.
        Args:
            data (obj:`List(str)`): The batch data whose each element is a raw text.
            tokenizer(obj:`PretrainedTokenizer`): This tokenizer inherits from :class:`~paddlenlp.transformers.PretrainedTokenizer`
                which contains most of the methods. Users should refer to the superclass for more information regarding methods.
        Returns:
            results(obj:`dict`): All the feature vectors.
        """

        examples = []
        for text in data:
            input_ids, segment_ids = convert_example_recall_infer(text, tokenizer)
            examples.append((input_ids, segment_ids))

        batchify_fn = lambda samples, fn=Tuple(
            Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype='int64'),  # input
            Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype='int64'),  # segment
        ): fn(samples)

        input_ids, segment_ids = batchify_fn(examples)
        self.input_handles[0].copy_from_cpu(input_ids)
        self.input_handles[1].copy_from_cpu(segment_ids)
        self.predictor.run()
        logits = self.output_handle.copy_to_cpu()
        return logits

    def predict(self, data, tokenizer):
        """
        Predicts the data labels.
        Args:
            data (obj:`List(str)`): The batch data whose each element is a raw text.
            tokenizer(obj:`PretrainedTokenizer`): This tokenizer inherits from :class:`~paddlenlp.transformers.PretrainedTokenizer`
                which contains most of the methods. Users should refer to the superclass for more information regarding methods.
        Returns:
            results(obj:`dict`): All the predictions probs.
        """

        examples = []
        for idx, text in enumerate(data):
            input_ids, segment_ids = convert_example_recall_infer({idx: text[0]}, tokenizer)
            title_ids, title_segment_ids = convert_example_recall_infer({
                idx: text[1]
            }, tokenizer)
            examples.append(
                (input_ids, segment_ids, title_ids, title_segment_ids))

        batchify_fn = lambda samples, fn=Tuple(
            Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype='int64'),  # input
            Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype='int64'),  # segment
            Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype='int64'),  # segment
            Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype='int64'),  # segment
        ): fn(samples)


        query_ids, query_segment_ids, title_ids, title_segment_ids = batchify_fn(
            examples)
        self.input_handles[0].copy_from_cpu(query_ids)
        self.input_handles[1].copy_from_cpu(query_segment_ids)
        self.predictor.run()
        query_logits = self.output_handle.copy_to_cpu()

        self.input_handles[0].copy_from_cpu(title_ids)
        self.input_handles[1].copy_from_cpu(title_segment_ids)
        self.predictor.run()
        title_logits = self.output_handle.copy_to_cpu()

        result = [
            float(1 - spatial.distance.cosine(arr1, arr2))
            for arr1, arr2 in zip(query_logits, title_logits)
        ]
        return result

In [ ]:
model_dir = './output/recall'
# device='gpu'
device='cpu'
max_seq_length=64
use_tensorrt = False
batch_size =32
precision = 'fp32'
cpu_threads = 1
enable_mkldnn =False
predictor = RecallPredictor(model_dir, device, max_seq_length,
                          batch_size, use_tensorrt, precision,
                          cpu_threads, enable_mkldnn)


id2corpus = {0: '国有企业引入非国有资本对创新绩效的影响——基于制造业国有上市公司的经验证据'}
corpus_list = [{idx: text} for idx, text in id2corpus.items()]
res = predictor.extract_embedding(corpus_list, tokenizer)
print('抽取向量')
print(res.shape)
print(res)


In [ ]:
corpus_list = [['中西方语言与文化的差异', '中西方文化差异以及语言体现中西方文化,差异,语言体现'],
                   ['中西方语言与文化的差异', '飞桨致力于让深度学习技术的创新与应用更简单']]
res = predictor.predict(corpus_list, tokenizer)
print('计算相似度')
print(res)